In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from particle_creation import par_create_2d,par_to_array
from mass_asign import cic_density_2d
from gravty import grav_from_den_2d
from acceleration import cic_acc_2d

In [10]:
ng = 16
num_par = 10

pars_pos = (np.random.rand(num_par, 2)*ng).astype(np.float32)
pars_vel = (np.random.rand(num_par, 2)).astype(np.float32)
# pars_mass = np.random.rand(num_par).astype(np.float32)*100
pars_mass = np.ones_like(pars_pos[:,0]).astype(np.float32)

# pars_pos = np.array([[0.1,11.1],[0.1,10.1]]).astype(np.float32)
# pars_vel = np.array([[10,0],[0,0]]).astype(np.float32)
# # pars_mass = np.ones_like(pars_pos[:,0]).astype(np.float32)
# pars_mass = np.array((0.00001,1)).astype(np.float32)

# pars_df = pd.DataFrame({'pos_y': pars_pos[:,0], 'pos_x': pars_pos[:,1],
#                         'vel_y': pars_vel[:,0], 'vel_x': pars_vel[:,1],
#                         'mass': pars_mass})
pars_df = pd.DataFrame({'pos_x': pars_pos[:,0], 'pos_y': pars_pos[:,1],
                        'vel_x': pars_vel[:,0], 'vel_y': pars_vel[:,1],
                        'mass': pars_mass,'id':np.arange(len(pars_pos))})
par_list = par_create_2d(pars_df)
pars_df

,pos_x,pos_y,vel_x,vel_y,mass,id
0,0.350576,3.430965,0.808997,0.506552,1.0,0
1,14.395676,12.965835,0.845885,0.819810,1.0,1
2,5.548877,4.422800,0.342430,0.203555,1.0,2
3,11.355205,14.948262,0.750364,0.444270,1.0,3
4,10.715922,3.099905,0.380161,0.782910,1.0,4
5,6.840656,1.537538,0.125448,0.748346,1.0,5
6,14.069949,12.927457,0.897670,0.695787,1.0,6
7,15.314313,12.856743,0.011014,0.598676,1.0,7
8,8.463495,2.529345,0.441864,0.945324,1.0,8
9,6.744768,0.730747,0.958834,0.370960,1.0,9


In [ ]:
dt = 0.1
# Define a colormap
# cmap = plt.cm.get_cmap('viridis', 10) 
fig, ax = plt.subplots(1, 1, figsize=(6, 5))
ax.set_ylim(0, ng)
for step in range(5):
     
    for par in par_list:
        # Fitrst step:
        par.pos += par.vel * 0.5 * dt
        # Boundary condition
        par.periodic(ng)
        
    dens = cic_density_2d(pars=par_list, ng=ng, h=1) ## Update density field
    gravity = grav_from_den_2d(dens)    ## Update gravity field
    cic_acc_2d(pars=par_list, ng=ng, gravity=gravity, h=1)  ## Update acceleration
    # print(par_list[0].acc, par_list[1].acc)
    # Second step:
    for par in par_list:
        par.vel += par.acc * dt
        par.pos += par.vel * 0.5 * dt
        # Boundary condition
        par.periodic(ng)
    # Plot
    
    for par in par_list:
        # color = cmap(par.id % 10) 
        im = ax.scatter(par.pos[0], par.pos[1], s=3, color=par.id)
        ax.set_xlim(0, ng)
        ax.set_ylim(0, ng)
# Set the colorbar
# norm = plt.Normalize(0, 9)  # Normalize the colormap to the range [0, 9]
# sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
# sm.set_array([])
# cbar = plt.colorbar(sm, ax=ax)
# cbar.set_label('Particle ID')

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 5))
for ax in axes.flatten():
    ax.set_xlim(0, ng)
    ax.set_ylim(0, ng)
im1 = axes[0].pcolormesh(acc_x)
cb1 = plt.colorbar(im1)
im2 = axes[1].pcolormesh(acc_x1)
cb2 = plt.colorbar(im2)